1. 对于数据集中每个句子，进行一次分词，得到数据格式 `情绪类别` `句子的词序列` `句子`
2. 对于每一种情绪，把训练集中的所有数据进行处理，得到一个data{key,value}的序列，其中key为句子中的不重复的词，value为该词在所有推文中出现的次数，计算data{key,value}序列每个词的先验$$P(w_i|c_j) = \frac{data\{w_i\}+1}{\sum_{q=1}^n data\{w_q\}+n}$$

首先读取数据 返回两个列表 label,sentence 列表中一一对应

**label sentences**

In [1]:
import math
import os.path
import random

import pandas as pd

filename = os.path.join(os.getcwd(), "sentiment_data/simplifyweibo_4_moods.csv")
with open(filename, "r", encoding="utf8") as f:
    csvData = pd.read_csv(f)
    csvKeys = csvData.keys()
    label = list(csvData.get(csvKeys[0]))
    sentences = list(csvData.get(csvKeys[1]))

allData = [{'label': str(label[i]).strip(), 'sentence': str(sentences[i]).strip()} for i in range(len(label))]
# 对allData随机洗牌
random.shuffle(allData)
# 抽取测试集、训练集出来
transData = allData[:int(len(allData) * 0.95)]
testData = allData[len(transData):]
# 训练集合
label = [i['label'] for i in transData]
sentences = [i['sentence'] for i in transData]

# 对读出来的数据进行拆分

然后 分词工具分词
**words**

In [2]:
import jieba
import jieba.posseg as pseg
from ltp import LTP

ltp = LTP()


def get_jieba_word(s):
    cnt = 0
    for i in s:
        # cnt += 1
        # if cnt % 100 == 0:
        #     print(cnt)
        # seg, hidden = ltp.seg([str(i)])
        # yield seg[0]
        # jieba
        yield jieba.lcut(str(i), use_paddle=True)
        # 下面这个会返回词性
        # yield pseg.lcut(str(i), use_paddle=True)


# 分词
words = list(get_jieba_word(sentences))


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/k9/8l5wkt157713f72dthyknqn00000gn/T/jieba.cache
Loading model cost 0.673 seconds.
Prefix dict has been built successfully.


然后开始统计词
**dataSet**

In [3]:
dataSet = {'0': {'keys': {}, 'count': 0, 'key_num': 0}, '1': {'keys': {}, 'count': 0, 'key_num': 0},
           '2': {'keys': {}, 'count': 0, 'key_num': 0}, '3': {'keys': {}, 'count': 0, 'key_num': 0}}
for i in range(len(label)):
    tmpData: dict = dataSet[label[i]]
    tmpWordList = words[i]
    for val in tmpWordList:
        if tmpData['keys'].__contains__(val):
            tmpData['keys'][val] += 1
        else:
            tmpData['keys'][val] = 1

处理停用词 去掉**dataSet**中的停用词

**stopWords** 删除停用词并统计后的**dataSet**

In [4]:
# 读取停用词
# with open(os.path.join(os.getcwd(), "stop_words.txt"), "r", encoding="utf8") as f:
#     stopWords = f.readlines()
#     stopWords = [str(i).strip() for i in stopWords]

# 读取情感词
with open(os.path.join(os.getcwd(), "sentiment_words.txt"), "r", encoding="utf8") as f:
    sentimentWords = f.readlines()
    sentimentWords = [str(i).strip() for i in sentimentWords]

# 删除集合中的停用词
for key in dataSet.keys():
    tmpSubKey = dataSet[key]['keys'].copy().keys()
    for subKey in tmpSubKey:
        # if subKey in stopWords:
        #     del dataSet[key]['keys'][subKey]
        if subKey not in sentimentWords:
            del dataSet[key]['keys'][subKey]
        else:
            dataSet[key]['count'] += dataSet[key]['keys'][subKey]
            dataSet[key]['key_num'] += 1

开始进行预测

In [5]:
import jieba
import math


# 定义计算公式
def predict(s: str, trainedData: dict):
    # seg, hidden = ltp.seg([str(s)])
    # preWords = seg[0]
    preWords = jieba.lcut(s, use_paddle=True)
    preResult = {}
    maxKey = None
    # 避免数值过小 矫正
    shiftNum = [trainedData[i]['count'] for i in trainedData.keys()][0] / len(trainedData.keys())
    # 对于每一个情绪类别
    for i in trainedData.keys():
        # 该情绪类别下的预测值 初始化为10000 放大一点
        preIValue = 1
        cnt = 0
        # 对于每一个预测句子中的词
        for w in preWords:
            # 如果不是停用词
            # if w not in stopWords:
            if w in sentimentWords:
                cnt += 1
                # 如果该情绪下有过这个词 直接计算
                if trainedData[i]['keys'].__contains__(w):
                    preIValue *= (trainedData[i]['keys'][w] + 1) * 1 / (
                            trainedData[i]['count'] + trainedData[i]['key_num'])

                # 该情绪下无这个词 默认为 1/kenLen
                else:
                    preIValue *= 1 / shiftNum
                    # preIValue *= 1 / trainedData[i]['key_num']
                    # preIValue *= 0.5
            # preIValue = 1 / (1 + math.exp(-preIValue))
        if cnt == 0:
            return maxKey, preResult
        preResult[i] = preIValue
        if maxKey is None or preIValue > preResult[maxKey]:
            maxKey = i
    return maxKey, preResult

# # 每计算1/10 test集合的句子 输出目前的准确度
# testDataLen = len(testData)
# preRightNum = 0
#
# precision = 0
# recall = 0
# f_measure = 0
# for i in range(testDataLen):
#     if i % (testDataLen // 10) == 0:
#         pass

In [6]:
# tes
num = 0
delNum = 0
cntNum = 0
for i in testData:
    cntNum += 1
    if cntNum % 100 == 0:
        print(num / (cntNum - delNum))
    pi, pd = predict(i['sentence'], dataSet)
    if pi is None:
        delNum += 1
        continue
    if i['label'] == pi:
        # print(i)
        # print(pi, pd)
        num += 1
print(num / (len(testData) - delNum))


0.45569620253164556
0.4409937888198758
0.43333333333333335
0.4444444444444444
0.44029850746268656
0.4451345755693582
0.4376130198915009
0.4464
0.4507845934379458
0.45302445302445304
0.45838218053927315
0.4536637931034483
0.45779543197616684
0.45
0.44232437120555074
0.4461538461538462
0.4484848484848485
0.4520743919885551
0.45275322909585314
0.4567741935483871
0.456
0.45368731563421827
0.4535211267605634
0.4540540540540541
0.45809474232170744
0.4625
0.4634382566585956
0.4630495790458372
0.4624773960216998
0.4595893403232853
0.46104995766299744
0.4633744855967078
0.46255506607929514
0.46153846153846156
0.4624952883528081
0.4630307467057101
0.460634128963306
0.46249567922571727
0.4616161616161616
0.4600197433366239
0.4600064246707356
0.45960389814523733
0.458754983134008
0.4591684116063416
0.45760233918128657
0.45685714285714285
0.4579596412556054
0.4574876577070762
0.4567669172932331
0.45643590418531194
0.4566787003610108
0.4582701062215478
0.4599752168525403
0.45940690325717065
0.459995

test